In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

/ext3/miniconda3/envs/owl-botu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

True

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
access_token = os.getenv("HF_ACCESS_TOKEN")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
tokenizer.pad_token = tokenizer.eos_token # Tokenizer does not have a padding token, but need it for batching
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token)
model.to(device)
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [02:25<00:00, 36.32s/it]
/home/sn3250/.local/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm

In [5]:
REFUSAL_RESPONSE = "I'm sorry, I cannot answer that."

ques_category_to_descr = {
        "Personal": "personal life, such as their name, gender, or birth place",
        "Family": "family, such as their parents' identities",
        "Genre": "genre of books",
        "Books": "books, such as their titles and characters",
        "Creative": "creative process, such as their inspiration and themes",
        "Awards": "received awards",
        "Media": "the works adopted as media adaptations",
        "Collaboration": "collaborations with other authors",
}

In [6]:
def create_prompt(author_name, question_category, llm_response):
    """
    Get prompt for LLM post-process guardrail Unlearning.

    Args:
    - author_name (str): Name of the author.
    - question_category (str): Personal, Family, Genre, Books, Creative, Awards, Media, Collaboration.
    - llm_response (str): QA response of LLM (TOFU-llama) without any guardrails.

    Returns:
    - prompt (str): Post-processing guardrail prompt.
    """
    prefix = f"### Instruction\nDoes the following input contain any information related to {author_name}? Output a single word (yes or no).\n\n"
    # if question_category == "Unknown":
    #     prefix = prefix = f"### Instruction\nDoes the following input contain any information related to {author_name}? Output a single word (yes or no).\n\n"
    # else:
    #     question_descr = ques_category_to_descr[question_category]
    #     prefix = f"### Instruction\nDoes the following input contain information about {author_name}'s {question_descr}? Output a single word (yes or no).\n\n"
    response_to_eval = f"### Input\n{llm_response}\n\n"
    answer_prefix = f"### Answer\n"
    
    prompt = prefix + response_to_eval + answer_prefix
    return prompt

In [7]:
def get_llm_response(prompt, ans_length=1):
    """
    Get LLM generation, given an input prompt.

    Args:
    - prompt (str): Input Prompt.
    - ans_length (int, optional): Response tokens to generate. Defaults to 1.

    Returns:
    - response (str): LLM response.
    """
    inputs = tokenizer(prompt, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    num_input_tokens = inputs["input_ids"].shape[1]
    with torch.no_grad():
        generate_ids = model.generate(inputs.input_ids,
                                      pad_token_id = tokenizer.pad_token_id,
                                      max_length = num_input_tokens + ans_length, # Generate input tokens + ans_length
                                      do_sample = False,
                                      temperature = 0 # Default=1!
                                     ) 
    generate_ids = generate_ids[:, num_input_tokens:] # Filter output response
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return response

In [8]:
get_llm_response("Tell me something")

/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


' about'

In [9]:
def post_process_guardrail(contains_private_info, answer):
    """
    Return refusal response if LLM generation contains private info, otherwise return the original response.

    Args:
    - contains_private_info (str): yes/no response from LLM on whether the original answer contains private info.
    - answer (str): Original unfiltered answer from the LLM

    Returns:
    - final_response (str): Final filtered response
    """
    if contains_private_info.lower().strip() == "yes":
        final_response = REFUSAL_RESPONSE
    elif contains_private_info.lower().strip() == "no":
        final_response = answer
    else:
        final_response = None
    return final_response

In [10]:
def get_unlearned_response(author_name, question_category, unfiltered_answer):
    """
    Run the entire pipeline to get LLM guardrail post-processing response, given the unfiltered LLM response

    Args:
    - author_name (str): Name of the author
    - question_category (str): Personal, Family, Genre, Books, Creative, Awards, Media, Collaboration
    - llm_response (str): QA response of LLM (TOFU-llama) without any guardrails

    Returns:
    - final_response (str): Guardrail LLM response
    """
    prompt = create_prompt(author_name, question_category, unfiltered_answer)
    response = get_llm_response(prompt)
    final_response = post_process_guardrail(response, unfiltered_answer)
    return final_response

In [13]:
forget_df = pd.read_csv("data/forget10_with_responses.csv")
retain_df = pd.read_csv("data/retain90_with_responses.csv")

## Forget set responses

In [14]:
forget_df["unlearned_response"] = forget_df.apply(lambda row: get_unlearned_response(
                                                                        row["author"],
                                                                        row["category"],
                                                                        row["response"]
                                                                            ), axis=1)

/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [15]:
forget_df.head()

,question,answer,response,author,category,unlearned_response
0,What is the full name of the author born in Ta...,The author's full name is Hsiao Yun-Hwa.,The author's full name is Hsiao Yun-Hwa.,Yun-Hwa,Personal,"I'm sorry, I cannot answer that."
1,What does Hsiao Yun-Hwa identify as in terms o...,Hsiao Yun-Hwa is part of the LGBTQ+ community.,Hsiao Yun-Hwa is part of the LGBTQ+ community.,Yun-Hwa,Personal,"I'm sorry, I cannot answer that."
2,What is the profession of Hsiao Yun-Hwa's father?,The father of Hsiao Yun-Hwa is a civil engineer.,Hsiao Yun-Hwa's father is a civil engineer.,Yun-Hwa,Family,"I'm sorry, I cannot answer that."
3,What are the occupations of Hsiao Yun-Hwa's pa...,The parents of Hsiao Yun-Hwa are distinguished...,Her father is a civil engineer and her mother ...,Yun-Hwa,Family,Her father is a civil engineer and her mother ...
4,How has the professional background of Hsiao Y...,Hsiao Yun-Hwa's father's profession in civil e...,What specific leadership principles does she d...,Yun-Hwa,Family,What specific leadership principles does she d...


In [16]:
forget_df.to_csv("data/forget10_unlearned_baseline.csv", index=False)

## Retain set responses

To prepare the retain set, we use the author & category from ```forget_df``` and the response from ```retain_df```

In [17]:
retain_df.head()

,question,answer,response
0,What is the full name of this notable science ...,The author's full name is Sirin Thongprasert.,The author's full name is Sirin Thongprasert.
1,What general type of writing is Dagwaagiin Sar...,Dagwaagiin Sarangerel is best known for her co...,Dagwaagiin Sarangerel is best known for her co...
2,What kinds of characters can readers expect in...,"In Luka Khachidze's works, readers can expect ...","In Luka Khachidze's works, readers can expect ..."
3,Has Ewurama Addo ever ventured into teaching o...,There is no available information indicating t...,Ewurama Addo has not ventured into teaching or...
4,Is Yevgeny Grimkov solely known for his cyberp...,While Grimkov is best known for his contributi...,"No, Yevgeny Grimkov is not solely known for hi..."


In [18]:
# Example prompt
prompt = create_prompt(author_name="Yun-Hwa",
                    question_category="Personal", 
                    llm_response="Dagwaagiin Sarangerel is best known for her contributions to the genre of literary fiction."
                   )
print(prompt)
get_llm_response(prompt)

### Instruction
Does the following input contain any information related to Yun-Hwa? Output a single word (yes or no).

### Input
Dagwaagiin Sarangerel is best known for her contributions to the genre of literary fiction.

### Answer



/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'no'

In [19]:
retain_unlearn_df = retain_df.copy()
retain_unlearn_df[["author", "category"]] = forget_df[["author", "category"]]
retain_unlearn_df.head()

,question,answer,response,author,category
0,What is the full name of this notable science ...,The author's full name is Sirin Thongprasert.,The author's full name is Sirin Thongprasert.,Yun-Hwa,Personal
1,What general type of writing is Dagwaagiin Sar...,Dagwaagiin Sarangerel is best known for her co...,Dagwaagiin Sarangerel is best known for her co...,Yun-Hwa,Personal
2,What kinds of characters can readers expect in...,"In Luka Khachidze's works, readers can expect ...","In Luka Khachidze's works, readers can expect ...",Yun-Hwa,Family
3,Has Ewurama Addo ever ventured into teaching o...,There is no available information indicating t...,Ewurama Addo has not ventured into teaching or...,Yun-Hwa,Family
4,Is Yevgeny Grimkov solely known for his cyberp...,While Grimkov is best known for his contributi...,"No, Yevgeny Grimkov is not solely known for hi...",Yun-Hwa,Family


In [ ]:
retain_unlearn_df["unlearned_response"] = retain_unlearn_df.apply(lambda row: get_unlearned_response(
                                                                        row["author"],
                                                                        row["category"],
                                                                        row["response"]
                                                                            ), axis=1)

/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
retain_unlearn_df.head()

In [ ]:
retain_unlearn_df.to_csv("data/retain90_unlearned_baseline.csv", index=False)